In [3]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
sys.path.append('..')

import utilities.functions as functions


from utilities.functions import (
    summary,
    matriz_migracao,
    save_parquet,
    gerar_resumo_decis,
    resumo_coorte_ativa,
    cria_base_decil_wide

)

In [4]:
df_pub_u = pd.read_parquet("../../dados/gold/df_pub_un.parquet")

Sumarizacao na visao cliente - mes

In [35]:
summary(df_pub_u)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
customer_id,1428530,806156,2c031493b15f0c727eb1b586bbe975de50311720439515...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
origin_platform,1428529,4,ANDROID,599328,NaN,NaN,NaN,NaN,NaN,NaN,NaN
is_target,1428530,2,target,799448,NaN,NaN,NaN,NaN,NaN,NaN,NaN
order_created_month,1428530.0,NaN,NaN,NaN,5.436391,5.39618,1.0,1.0,1.0,12.0,12.0
num_pedidos_mes,1428530.0,NaN,NaN,NaN,2.864955,3.497821,1.0,1.0,2.0,3.0,157.0
num_pedidos_hist,1428530.0,NaN,NaN,NaN,5.414977,6.429829,1.0,2.0,3.0,6.0,239.0
total_amount_mes,1428530.0,NaN,NaN,NaN,137.183218,264.907875,0.0,39.9,72.4,152.5,140338.9
ticket_medio,1428530.0,NaN,NaN,NaN,48.017112,92.438889,0.0,29.475,41.0,56.97,70169.45


Clientes mes x target

In [ ]:
clientes_mes = (df_pub_u.groupby(['order_created_month', 'is_target'])
              ['customer_id']
              .nunique()
              .reset_index(name='numero_clientes_distintos'))
clientes_mes

 Matriz de migracao - cliente

In [ ]:
matriz_migracao(df_pub_u,mes_0=12,mes_1=1)

Dado o montante de clientes novos em janeiro a analise sera feita basead em cliente ambos os meses e cliente somente em janeiro

In [5]:
id_both_monht=df_pub_u[df_pub_u['order_created_month']==12]['customer_id'].unique()
publico_janeiro_dezembro = df_pub_u[df_pub_u['customer_id'].isin(id_both_monht)].reset_index(drop=True)
publico_janeiro = df_pub_u[~df_pub_u['customer_id'].isin(id_both_monht)].reset_index(drop=True)

In [ ]:
publico_janeiro.head()

In [ ]:
save_parquet(publico_janeiro_dezembro, "gold", "publico_janeiro_dezembro.parquet")


Pedidos por mes - target

In [ ]:
pedidos_total = (
    publico_janeiro_dezembro.groupby(['is_target','order_created_month'])['num_pedidos_mes']
      .sum()
      .reset_index(name='numero_de_pedidos_total')
)

pedidos_total

Distruibuicao do total amount por mes

In [ ]:
df_stats_mes = publico_janeiro_dezembro.groupby(['order_created_month', 'is_target'])['total_amount_mes'].agg(
    Média=('mean'),
    Mediana=('median'),
    Mínimo=('min'),
    Máximo=('max'),
    Desvio_Padrão=('std')
).round(2)
df_stats_mes

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(
    data=publico_janeiro_dezembro,
    y='total_amount_mes',
    x='order_created_month',  
    hue='is_target',          
    
)

plt.title('Distribuição do Valor Gasto por Mês e Grupo (Target vs Control)')
plt.ylabel('Total Amount (R$)')
plt.xlabel('Mês')
plt.legend(title='Grupo')
plt.show()

Observa-se grande numero de outliers, verificar a distribuicao removendo outliers.

In [ ]:
publico_janeiro_dezembro['total_amount_mes'].describe().round(2)


In [ ]:
red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')
publico_janeiro_dezembro['total_amount_mes'].plot(kind='box', xlim=(0, 500), vert=False, flierprops=red_square, figsize=(16,2))

In [ ]:
distance = 1.5 * (np.nanpercentile(publico_janeiro_dezembro['total_amount_mes'], 75) - np.nanpercentile(publico_janeiro_dezembro['total_amount_mes'], 25))
lim_sup=distance + np.nanpercentile(publico_janeiro_dezembro['total_amount_mes'], 75)

In [ ]:
pb_drop = publico_janeiro_dezembro.drop(
    publico_janeiro_dezembro[
        (publico_janeiro_dezembro['total_amount_mes'] == 0) |
        (publico_janeiro_dezembro['total_amount_mes'] > lim_sup)
    ].index,
    axis=0
)

In [ ]:
pb_drop['total_amount_mes'].describe().round(2)

In [ ]:
red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')
pb_drop['total_amount_mes'].plot(kind='box', xlim=(0, 400), vert=False, flierprops=red_square, figsize=(16,2))

In [ ]:
df_decil,decil_dict=cria_base_decil_wide(publico_janeiro_dezembro,
    mes_0=12,  
    mes_1=1,  
    col_cliente="customer_id",
    col_mes="order_created_month",
    col_pedidos="num_pedidos_mes",
    col_valor="total_amount_mes",
    col_target="is_target",  
    n_decis=10,
    prefixo_decil="decil",
    prefixo_pedidos="num_pedidos_mes",
    prefixo_valor="total_amount_mes"
)

Decil corrigido

In [ ]:
# Tudo em um só lugar - sem copiar tabelas
publico_janeiro_dezembro['decil_mes'] = publico_janeiro_dezembro['total_amount_mes'].apply(marca)

# Separar e renomear em uma linha
df_wide = (
    publico_janeiro_dezembro
    .pivot(index=['customer_id', 'is_target'], columns='order_created_month', values=['total_amount_mes', 'num_pedidos_mes', 'decil_mes'])
    .reset_index()
)
df_wide.columns = ['customer_id', 'is_target', 'total_amount_mes_12', 'total_amount_mes_1', 'num_pedidos_mes_12', 'num_pedidos_mes_1', 'decil_12', 'decil_1']

# Calcular estatísticas
clientes_por_decil_target = (
    df_wide
    .groupby(['decil_12', 'decil_1', 'is_target'])
    .agg(total_clientes=('customer_id', 'nunique'),
         avg_amount_12=('total_amount_mes_12', 'mean'),
         avg_amount_1=('total_amount_mes_1', 'mean'),
         avg_pedidos_12=('num_pedidos_mes_12', 'mean'),
         avg_pedidos_1=('num_pedidos_mes_1', 'mean'))
    .assign(
        ticket_medio_12=lambda df: df['avg_amount_12'] / df['avg_pedidos_12'],
        ticket_medio_1=lambda df: df['avg_amount_1'] / df['avg_pedidos_1']
    )
    .round(2)
    .reset_index()
)

In [ ]:
publico_janeiro_dezembro[publico_janeiro_dezembro['order_created_month']==12]['decil'].unique()

In [ ]:
#import os
#os.makedirs('../dados/gold', exist_ok=True)

decil_dict_json = {k: list(v) for k, v in decil_dict.items()}
with open('dados/gold/decil_dictionary.json', 'w') as f:
    json.dump(decil_dict_json, f, indent=2)

In [ ]:
decil_dict_json

In [ ]:
publico_janeiro_dezembro['decil_mes_2'] = publico_janeiro_dezembro['total_amount_mes'].apply(
    lambda x: next(
        (decil for decil, faixa in sorted(decil_dict_json.items(), reverse=True) 
         if x >= faixa[0]),
        "Sem classificação"  # default se não encontrar
    )
)

In [ ]:
publico_janeiro_dezembro[publico_janeiro_dezembro['order_created_month']==1]['total_amount_mes'].describe()

In [ ]:
# Pivot para ter colunas separadas por mês
df_pivot = publico_janeiro_dezembro.pivot_table(
    index=['customer_id', 'is_target'],  # Identificadores únicos
    columns='order_created_month',                          # Mês vira coluna
    values=['num_pedidos_mes', 'total_amount_mes', 'ticket_medio', 'decil_mes_2', 'num_pedidos_hist'],
    aggfunc='first'  # Pega o primeiro valor (já que é único por mês)
).reset_index()

# Limpar o multi-index das colunas
df_pivot.columns = [f'{col[0]}_{col[1]}' if col[1] != '' else col[0] 
                   for col in df_pivot.columns]

In [ ]:
save_parquet(df_pivot, "gold", "df_decil.parquet")

In [ ]:
decil_summary=gerar_resumo_decis(df_decil)
decil_summary

In [ ]:
os.makedirs('../Resultados', exist_ok=True)

decil_summary.to_csv('../Resultados/decil_summary.csv', index=False)



Numero de pedidos em Janeiro e Dezembro para o quem esteve em ambos meses

In [ ]:

pedidos_hist=resumo_coorte_ativa(publico_janeiro_dezembro,mes_coorte_inicio=12,mes_coorte_fim=1)
pedidos_hist

In [ ]:
pedidos_hist.to_csv('../Resultados/pedidos_hist.csv', index=False)

Correcao decil